In [1]:
import requests
from bs4 import BeautifulSoup

retreiving the URL and creating a Beautiful soup object

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto"

data  = requests.get(url).text

In [3]:
soup = BeautifulSoup(data, 'html5lib')

**Firstly create a list**  

 **Later after finding the table and table data  create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood.**

**As postal code contains upto 3 characters extract that using tablerow.p.text**

 **Next use split ,strip and replace functions for getting Borough and Neighborhood information.**.

 **Append to the list**  

 **Create a dataframe with list**

In [47]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if type(row.span) != None :
      print(row)
      if row.span.text=='Not assigned':
          pass
      else:
          cell['PostalCode'] = row.p.text[:3]
          cell['Borough'] = (row.span.text).split('(')[0]
          cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
          table_contents.append(cell)

# print(table_contents)
df=pandas.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

<td style="text-align: left; vertical-align: top;">
<ul><li><a href="/wiki/Alexandra_Park,_Toronto" title="Alexandra Park, Toronto">Alexandra Park</a></li>
<li><a href="/wiki/The_Annex" title="The Annex">The Annex</a></li>
<li><a href="/wiki/Baldwin_Village" title="Baldwin Village">Baldwin Village</a></li>
<li><a href="/wiki/Cabbagetown,_Toronto" title="Cabbagetown, Toronto">Cabbagetown</a></li>
<li><a href="/wiki/CityPlace,_Toronto" title="CityPlace, Toronto">CityPlace</a></li>
<li><a href="/wiki/Chinatown,_Toronto" title="Chinatown, Toronto">Chinatown</a></li>
<li><a href="/wiki/Church_and_Wellesley" title="Church and Wellesley">Church and Wellesley</a></li>
<li><a href="/wiki/Corktown,_Toronto" title="Corktown, Toronto">Corktown</a></li>
<li><a href="/wiki/Discovery_District" title="Discovery District">Discovery District</a></li>
<li><a href="/wiki/Distillery_District" title="Distillery District">Distillery District</a></li>
<li><a href="/wiki/Toronto_Entertainment_District" title="

AttributeError: ignored

In [48]:
dataset = pandas.read_csv('Geospatial_Coordinates.csv')

In [49]:
neighborhoods = pandas.merge(dataset, df, on='Code')

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Create a map of Tororo with neighborhoods superimposed on top.

In [ ]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(datasetfinal['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Toronto. So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [ ]:
toronto_data = neighborhoods[neighborhoods['Borough'] == 'Toronto'].reset_index(drop=True)
toronto_data.head()

## Analyze Each Neighborhood

let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = neighborhood.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toront_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toront_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters